### imports

In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

import emission.storage.timeseries.abstract_timeseries as esta
import emission.storage.decorations.trip_queries as esdtq
import performance_eval
import data_wrangling


### load data

In [ ]:
all_users = esta.TimeSeries.get_uuid_list()

confirmed_trip_df_map = {}
labeled_trip_df_map = {}
expanded_labeled_trip_df_map = {}
expanded_all_trip_df_map = {}
for u in all_users:
    ts = esta.TimeSeries.get_time_series(u)
    ct_df = ts.get_data_df("analysis/confirmed_trip")

    confirmed_trip_df_map[u] = ct_df
    labeled_trip_df_map[u] = esdtq.filter_labeled_trips(ct_df)
    expanded_labeled_trip_df_map[u] = esdtq.expand_userinputs(
        labeled_trip_df_map[u])
    expanded_all_trip_df_map[u] = esdtq.expand_userinputs(
        confirmed_trip_df_map[u])

### run DBSCAN at varying distance thresholds

In [ ]:
try:
    all_results_df = pd.read_csv('DBSCAN_vary_distance_threshold.csv')
except:
    param_grid = {
        'DBSCAN': {
            'SVM': [False],
            'min_samples': [1],
            'gamma': [0.05],
            'C': [1],
            'size_thresh': [1],
            'purity_thresh': [1],
        },
    }
    radii = [50, 100, 150, 200]

    all_results_df = performance_eval.run_eval_cluster_metrics(
        expanded_labeled_trip_df_map,
        user_list=all_users,
        radii=radii,
        loc_type='end',
        algs=['DBSCAN'],
        param_grid=param_grid,
        n_iter=1,
        random_state=42)

    all_results_df = data_wrangling.expand_df_dict(all_results_df, 'params')
    all_results_df.to_csv('DBSCAN_vary_distance_threshold.csv', index=False)

### plot resulting cluster metrics

In [ ]:
plt.style.use("default")
fig, axs = plt.subplots(1,
                        3,
                        figsize=(9, 4),
                        gridspec_kw={'width_ratios': [4, 2, 2]})

purities = []
req_pcts = []

radii = [50, 100, 150, 200]

for i in range(len(radii)):
    r = radii[i]
    results = all_results_df.loc[all_results_df['radius'] == r]
    purity = results.loc[:, 'purity']
    req_pct = results.loc[:, 'n_clusters'] / results.loc[:, 'n_trips']
    purities += [purity]
    req_pcts += [req_pct]
    axs[0].scatter(x=purity,
                   y=req_pct,
                   s=12,
                   color=plt.cm.tab10(i),
                   alpha=0.8,
                   edgecolors='white',
                   label=r)
axs[0].set_aspect('equal', adjustable='box')
axs[0].legend(title='Distance Threshold (m)')._legend_box.align = 'left'

axs[0].set_ylabel('Request Percentage')
axs[0].set_xlabel('Mean Cluster Purity by User')
axs[0].set_xlim(-0.05, 1.05)
axs[0].set_ylim(-0.05, 1.05)
axs[1].set_ylim(-0.05, 1.05)
axs[2].set_ylim(-0.05, 1.05)

axs[1].boxplot(purities)
axs[2].boxplot(req_pcts)

axs[1].set_xticklabels(radii)
axs[2].set_xticklabels(radii)

axs[1].set_ylabel('Mean Cluster Purity by User')
axs[2].set_ylabel('Request Percentage')

axs[1].set_xlabel('Distance Threshold (m)')
axs[2].set_xlabel('Distance Threshold (m)')

plt.tight_layout()
plt.show()


plot the same as above, but switch the XY axes for the boxplots

In [ ]:
plt.style.use("default")
fig = plt.figure(figsize=(9, 4))

gs = GridSpec(2, 7)
axs = []
axs.append(plt.subplot(gs[:, 0:3]))
axs.append(plt.subplot(gs[0, 3:7]))
axs.append(plt.subplot(gs[1, 3:7]))

purities = []
req_pcts = []

radii = [50, 100, 150, 200]

for i in range(len(radii)):
    r = radii[i]
    results = all_results_df.loc[all_results_df['radius'] == r]
    purity = results.loc[:, 'purity']
    req_pct = results.loc[:, 'n_clusters'] / results.loc[:, 'n_trips']
    purities += [purity]
    req_pcts += [req_pct]
    axs[0].scatter(x=purity,
                   y=req_pct,
                   s=12,
                   color=plt.cm.tab10(i),
                   alpha=0.8,
                   edgecolors='white',
                   label=r)
axs[0].set_aspect('equal', adjustable='box')
axs[0].legend(title='Distance Threshold (m)')._legend_box.align = 'left'

axs[0].set_ylabel('Request Percentage')
axs[0].set_xlabel('Mean Cluster Purity by User')
axs[0].set_xlim(-0.05, 1.05)
axs[0].set_ylim(-0.05, 1.05)
axs[1].set_xlim(-0.05, 1.05)
axs[2].set_xlim(-0.05, 1.05)

axs[1].boxplot(purities, vert=False)
axs[2].boxplot(req_pcts, vert=False)

axs[1].set_yticklabels(radii)
axs[2].set_yticklabels(radii)

axs[1].set_xlabel('Mean Cluster Purity')
axs[2].set_xlabel('Request Percentage')

axs[1].set_ylabel('Distance Threshold (m)')
axs[2].set_ylabel('Distance Threshold (m)')

plt.tight_layout()
plt.show()


this time we only want to compare: 
* DBSCAN+SVM, 100m
* DBSCAN, 100m
* DBSCAN, 50m

first, we need to run the algorith for DBSCAN+SVM.

In [ ]:
SVM_param_grid = {
    'DBSCAN': {
        'SVM': [True],
        'min_samples': [1],
        'gamma': [0.05],
        'C': [1],
        'size_thresh': [1],
        'purity_thresh': [1],
        # 'radii': [50, 100, 150]
    },
}
radii = [100]

SVM_results_df = performance_eval.run_eval_cluster_metrics(
    expanded_labeled_trip_df_map,
    user_list=all_users,
    radii=radii,
    loc_type='end',
    algs=['DBSCAN'],
    param_grid=SVM_param_grid,
    n_iter=1,
    random_state=42)

SVM_results_df = data_wrangling.expand_df_dict(SVM_results_df, 'params')
combined_results_df = pd.concat([all_results_df, SVM_results_df],
                                ignore_index=True)


now plot it:

In [ ]:
plt.style.use("default")
fig = plt.figure(figsize=(9, 4))

gs = GridSpec(2, 7)
axs = []
axs.append(plt.subplot(gs[:, 0:3]))
axs.append(plt.subplot(gs[0, 3:7]))
axs.append(plt.subplot(gs[1, 3:7]))

purities = []
req_pcts = []

radii = [50, 100]
conditions = [(100, True), (100, False), (50, False)] # (radius, SVM)

labels = ["DBSCAN+SVM, 100m", "DBSCAN, 100m", "DBSCAN, 50m"]

for i, c in enumerate(conditions):
    rad, svm = c
    results = combined_results_df.loc[(combined_results_df['radius'] == rad)
                                      & (combined_results_df['SVM'] == svm)]
    purity = results.loc[:, 'purity']
    req_pct = results.loc[:, 'n_clusters'] / results.loc[:, 'n_trips']
    purities += [purity]
    req_pcts += [req_pct]
    axs[0].scatter(x=purity,
                   y=req_pct,
                   s=12,
                   color=plt.cm.tab10(i),
                   alpha=0.8,
                   edgecolors='white',
                   label=labels[i])
axs[0].set_aspect('equal', adjustable='box')
axs[0].legend()

axs[0].set_ylabel('Request Percentage')
axs[0].set_xlabel('Mean Cluster Purity by User')
axs[0].set_xlim(-0.05, 1.05)
axs[0].set_ylim(-0.05, 1.05)
axs[1].set_xlim(-0.05, 1.05)
axs[2].set_xlim(-0.05, 1.05)

bp_purity = axs[1].boxplot(purities, vert=False)
bp_reqpct = axs[2].boxplot(req_pcts, vert=False)

for element in ['boxes', 'fliers', 'means', 'medians']: 
    # ignore 'whiskers', 'caps'
    for i, (el_pur, el_rp) in enumerate(zip(bp_purity[element], bp_reqpct[element])):
        plt.setp(el_pur, color=plt.cm.tab10(i))
        plt.setp(el_rp, color=plt.cm.tab10(i))

axs[1].set_yticklabels([])
axs[2].set_yticklabels([])

axs[1].set_xlabel('Mean Cluster Purity')
axs[2].set_xlabel('Request Percentage')

plt.tight_layout()
plt.show()

In [ ]:
import scipy
import numpy as np

In [ ]:
def func(x, a, b):
    return a * x + b

In [ ]:
plt.style.use("default")
fig, axs = plt.subplots(1,
                        4,
                        figsize=(9, 4),
                        sharex=True,
                        sharey=True)

purities = []
req_pcts = []

radii = [50, 100, 150, 200]

for i in range(len(radii)):
    r = radii[i]
    results = all_results_df.loc[all_results_df['radius'] == r].copy()
    results['req_pct'] = results.loc[:, 'n_clusters'] / results.loc[:, 'n_trips']
    results.sort_values(by='purity', inplace=True)
    hom = results.loc[:, 'homogeneity']
    mod_hom = results.loc[:, 'modified_homogeneity']
    purity = results.loc[:, 'purity']
    req_pct = results.loc[:, 'req_pct']
    purities += [purity]
    req_pcts += [req_pct]
    popt, pcov = scipy.optimize.curve_fit(func, purity, req_pct)
    print("After curve fitting, parameters are %s" % popt)
    axs[i].scatter(x=purity,
                   y=req_pct,
                   s=12,
                   alpha=0.8,
                   edgecolors='white')
    axs[i].plot(purity, func(purity, *popt), "r-")
    axs[i].set_title("Distance Threshold %d" % r)
    axs[i].set_xlabel('Mean Cluster Purity by User')

axs[0].set_ylabel('Request Percentage')
axs[0].set_xlim(-0.05, 1.05)
axs[0].set_ylim(-0.05, 1.05)

# fig.suptitle('DBSCAN Cluster Quality Across All Users')
plt.tight_layout()
plt.show()


In [ ]:
plt.style.use("default")
fig, axs = plt.subplots(1,
                        1,
                        figsize=(6, 6),
                        sharex=True,
                        sharey=True)

purities = []
req_pcts = []

radii = [50, 100, 150, 200]

for i in range(len(radii)):
    r = radii[i]
    results = all_results_df.loc[all_results_df['radius'] == r].copy()
    results['req_pct'] = results.loc[:, 'n_clusters'] / results.loc[:, 'n_trips']
    results.sort_values(by='purity', inplace=True)
    hom = results.loc[:, 'homogeneity']
    mod_hom = results.loc[:, 'modified_homogeneity']
    purity = results.loc[:, 'purity']
    req_pct = results.loc[:, 'req_pct']
    popt, pcov = scipy.optimize.curve_fit(func, purity, req_pct)
    print("After curve fitting, parameters are %s" % popt)
    axs.scatter(x=purity,
                   y=req_pct,
                   s=12,
                   alpha=0.8,
                   color=plt.cm.tab10(i),
                   edgecolors='white',
                   label=r)
    axs.plot(purity, func(purity, *popt), color=plt.cm.tab10(i))
    axs.set_title("Distance Threshold %d" % r)
    axs.set_xlabel('Mean Cluster Purity by User')

axs.set_ylabel('Request Percentage')
axs.set_xlim(-0.05, 1.05)
axs.set_ylim(-0.05, 1.05)
axs.legend(title='Distance Threshold (m)')._legend_box.align = 'left'

# fig.suptitle('DBSCAN Cluster Quality Across All Users')
plt.tight_layout()
plt.show()
